In [2]:
# Boosted Decision Tree Test File for AN-19-154 Work

# Coders: Nathan Suri, Caltech
# Date: September 24, 2019
# LPC LLP Group

# Description
# Study shower shape to improve signal/bkg discrimination

# Action Plan
# Begin developing simple BDT

# Notes/Conclusions
# @nasurijr: 

In [3]:
# User specifics
# Setups pwd location for data files and imports of special ROOT utilities

work_location = input("Username: ")
if work_location == 'nasurijr':
    pwd = '/nfshome/nasurijr/LLP_analysis/delayed_jet_analyzer/'
    
    # Sets display width
    from IPython.core.display import display, HTML
    display(HTML("<style>.container { width:85% !important; }</style>"))
    
# elif work_location == '<Insert Tier2 username here>':
#     pwd = '/home/cms/delayed_jet_analyzer/'

Username: nasurijr


# Imports

In [4]:
# Imports necessary utilities and modules

import ROOT as rt
import root_numpy as rtnp
import numpy as np
from matplotlib import pyplot as plt
from scipy.spatial.distance import cdist
from collections import Counter 
import datetime
import pytz

# Graph/histo utilities from ROOT
# Contained within the delayed_jet_analyzer repository
import sys
sys.path.append(pwd+'lib')
from histo_utilities import create_TH1D, create_TH2D, create_TGraph, std_color_list

# Used for extracting the TTree structure from each datafile
import os
import uproot

# Used for creating user-readable tables
from prettytable import PrettyTable

# Imports jet clustering algorithm (FastJet)
from pyjet import cluster

# Imports XGBoost BDT package
import xgboost as xgb

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from xgboost import XGBClassifier

donotdelete = []

Welcome to JupyROOT 6.18/02


In [5]:
# Setups dictionaries for storing data from MC/data ntuples
fpath = {}
tree = {}
NEvents = {}

data_path = pwd+'data/'

# Background Samples
fpath['WJetsToLNu'] = data_path + 'WJetsToLNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8_1pb_weighted.root'

# Signal Samples 
fpath['m55ct10m'] = data_path + 'WH_HToSSTobbbb_WToLNu_MH-125_MS-55_ctauS-10000_TuneCUETP8M1_13TeV-powheg-pythia8_1pb_weighted.root'

for k,v in fpath.items():
    print(str(datetime.datetime.now(pytz.timezone('US/Pacific'))))
    print(k, v)
    root_dir = uproot.open(v) 
    tree[k] = root_dir['MuonSystem']
    NEvents[k] = root_dir['NEvents'][1]

2019-09-25 16:42:17.311244-07:00
WJetsToLNu /nfshome/nasurijr/LLP_analysis/delayed_jet_analyzer/data/WJetsToLNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8_1pb_weighted.root
2019-09-25 16:42:17.409001-07:00
m55ct10m /nfshome/nasurijr/LLP_analysis/delayed_jet_analyzer/data/WH_HToSSTobbbb_WToLNu_MH-125_MS-55_ctauS-10000_TuneCUETP8M1_13TeV-powheg-pythia8_1pb_weighted.root


## Name TTree Objects

In [6]:
# Bookkeeping: Defines the TTrees from the read datafiles
# Names displayed in README.md table

T_m55ct10m = tree['m55ct10m']
T_wjets = tree['WJetsToLNu']

In [7]:
# Bookkeeping: Creates a dictionary for iterating over all of the datafiles and 
#              converting the relevant branches to numpy arrays
# Names displayed in README.md table

# data_trees = {'m55ct10m_wh_bbbb_minus': T_m55_ct10_minus, 'm55ct10m_wh_bbbb_plus': T_m55_ct10_plus, 'WJetsToLNu': T_wjets, 'm15ct10000mm': T_m15_ct10}

data_trees = {'m55ct10m': T_m55ct10m, 'WJetsToLNu': T_wjets}

# Define and Store Variables

In [8]:
# Variable Definitions
# Creates dictionaries for variables to be analyzed
# The dictionaries will contain the variable arrays for each datafile with a relevant key

# General Information
eventNum = {}
lumiNum = {}
weight = {}

# Standard CSC vars
nCsc = {}
csc_z = {}
csc_x = {}
csc_y = {}
csc_eta = {}
csc_phi = {}
csc_t = {}
csc_r = {}
nCsc_pop = {}

# CSC Vectors
csc_dir_x = {}
csc_dir_y = {}
csc_dir_z = {}

# Boolean Vetoes
nCsc_jet_veto = {}
nCsc_combo_veto = {}

# Shower Shape
csc_cluster_x = {}
csc_cluster_y = {}
csc_cluster_z = {}
csc_cluster_t = {}
csc_cluster_eta = {}
csc_cluster_phi = {}

In [9]:
print('Start: ' + str(datetime.datetime.now(pytz.timezone('US/Pacific'))))

for species, arbor in data_trees.items():
    
    # General Information
    eventNum[species] = arbor['evtNum'].array()
    lumiNum[species] = arbor['lumiSec'].array()
    weight[species] = arbor['weight'].array()
    
    # Standard CSC vars
    nCsc[species] = arbor['nCsc'].array()
    nCsc_pop[species] = arbor['cscClusterSize'].array()
    csc_z[species] = arbor['cscZ'].array()
    csc_x[species] = arbor['cscX'].array()
    csc_y[species] = arbor['cscY'].array()
    csc_t[species] = arbor['cscT'].array()
    csc_eta[species] = arbor['cscEta'].array()
    csc_phi[species] = arbor['cscPhi'].array()
        
    # CSC Vectors
    csc_dir_x[species] = arbor['cscDirectionX'].array()
    csc_dir_y[species] = arbor['cscDirectionY'].array()
    csc_dir_z[species] = arbor['cscDirectionZ'].array()
    
    # Boolean Vetoes
    nCsc_jet_veto[species] = arbor['nCsc_JetVetoCluster0p4_Me1112Veto'].array()
    nCsc_combo_veto[species] = arbor['nCsc_JetMuonVetoCluster0p4_Me1112Veto'].array()
    
    # Shower Shape
    csc_cluster_x[species] = arbor['cscClusterXSpread'].array().flatten()
    csc_cluster_y[species] = arbor['cscClusterYSpread'].array().flatten()
    csc_cluster_z[species] = arbor['cscClusterZSpread'].array().flatten()
    csc_cluster_t[species] = arbor['cscClusterTimeSpread'].array().flatten()
    csc_cluster_eta[species] = arbor['cscClusterEtaSpread'].array().flatten()
    csc_cluster_phi[species] = arbor['cscClusterPhiSpread'].array().flatten()
    
    print(species + ': ' +str(datetime.datetime.now(pytz.timezone('US/Pacific'))))

Start: 2019-09-25 16:42:18.122610-07:00
m55ct10m: 2019-09-25 16:42:18.240564-07:00
WJetsToLNu: 2019-09-25 16:42:24.867056-07:00


# BDT Implementation

## Convert to DMatrix Format

In [10]:
# Step 1: Combine signal and background arrays

csc_cluster_x['input'] = np.concatenate((csc_cluster_x['m55ct10m'], csc_cluster_x['WJetsToLNu']))
csc_cluster_y['input'] = np.concatenate((csc_cluster_y['m55ct10m'], csc_cluster_y['WJetsToLNu']))
csc_cluster_z['input'] = np.concatenate((csc_cluster_z['m55ct10m'], csc_cluster_z['WJetsToLNu']))
csc_cluster_t['input'] = np.concatenate((csc_cluster_t['m55ct10m'], csc_cluster_t['WJetsToLNu']))
csc_cluster_eta['input'] = np.concatenate((csc_cluster_eta['m55ct10m'], csc_cluster_eta['WJetsToLNu']))
csc_cluster_phi['input'] = np.concatenate((csc_cluster_phi['m55ct10m'], csc_cluster_phi['WJetsToLNu']))

In [11]:
# Step 2: Label signal and background data points

csc_cluster_label = np.concatenate((np.full(len(csc_cluster_x['m55ct10m']), True), np.full(len(csc_cluster_x['WJetsToLNu']), False)))

In [12]:
# Step 3: Gather all feature names

csc_cluster_features = ['cscClusterXSpread', 'cscClusterYSpread', 'cscClusterZSpread', 'cscClusterTimeSpread', 'cscClusterEtaSpread', 'cscClusterPhiSpread']

In [13]:
# Step 4: Construct 2D Numpy data array

data_bdt = np.dstack((csc_cluster_x['input'], csc_cluster_y['input'], csc_cluster_z['input'], csc_cluster_t['input'], csc_cluster_eta['input'], csc_cluster_phi['input'], csc_cluster_label))[0]
np.random.shuffle(data_bdt)

In [14]:
# Step 5: Define training and testing sets

data_train = data_bdt[:384552]
data_test = data_bdt[384552:]

In [15]:
# Step 6: Convert np.arrays into DMatrix format

# train = data_train[:,0:-1]
# test = data_test[:,0:-1]

train = xgb.DMatrix(data = data_train[:,0:-1],label=data_train[:,-1], missing=-999.0,feature_names=csc_cluster_features)
test = xgb.DMatrix(data = data_test[:,0:-1],label=data_train[:,-1], missing=-999.0,feature_names=csc_cluster_features)

/usr/local/lib/python3.6/site-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


## Set Hyperparameters and Train

In [16]:
param = {}

# Booster parameters
param['eta']              = 0.3 # learning rate
param['max_depth']        = 5  # maximum depth of a tree
param['subsample']        = 0.2 # fraction of events to train tree on
param['colsample_bytree'] = 0.5 # fraction of features to train tree on

# Learning task parameters
param['objective']   = 'binary:logistic' # objective function
param['eval_metric'] = 'error'           # evaluation metric for cross validation
param['nthreads'] = 4
param = list(param.items()) + [('eval_metric', 'logloss')] + [('eval_metric', 'rmse')]

num_trees = 50  # number of trees to make

In [17]:
print('Start: ' +str(datetime.datetime.now(pytz.timezone('US/Pacific'))))
booster = xgb.train(param,train,num_boost_round=num_trees)
print('End: ' +str(datetime.datetime.now(pytz.timezone('US/Pacific'))))

Start: 2019-09-25 16:42:26.660631-07:00
End: 2019-09-25 16:42:34.050878-07:00


In [ ]:
predictions = booster.predict(test)

In [ ]:
# plot all predictions (both signal and background)
plt.figure();
plt.hist(predictions,bins=np.linspace(0,1,50),histtype='step',color='darkgreen',label='All events');
# make the plot readable
plt.xlabel('Prediction from BDT',fontsize=12);
plt.ylabel('Events',fontsize=12);
plt.legend(frameon=False);

# # plot signal and background separately
# plt.figure();
# plt.hist(predictions[test.get_label().astype(bool)],bins=np.linspace(0,1,50),
#          histtype='step',color='midnightblue',label='signal');
# plt.hist(predictions[~(test.get_label().astype(bool))],bins=np.linspace(0,1,50),
#          histtype='step',color='firebrick',label='background');
# # make the plot readable
# plt.xlabel('Prediction from BDT',fontsize=12);
# plt.ylabel('Events',fontsize=12);
# plt.legend(frameon=False);